In [5]:
#Import Libraries
import pandas as pd
from datetime import datetime
import os
from sqlalchemy import create_engine, String, DateTime, Integer, text
from dotenv import load_dotenv

In [6]:
## Function to merge the csv files

def merge_csv(data_location, file_name):

    individual_data = []

    for filee in os.listdir(data_location):
        if filee.endswith('.csv'):
            file_path = os.path.join(data_location, filee)
            read_data = pd.read_csv(file_path, dtype={'season': str})
            individual_data.append(read_data)

    E0_Data = pd.concat(individual_data, ignore_index=True)
    ff_path = os.path.join(data_location, file_name)
    E0_Data.to_csv(ff_path, index=False)
    print(f'Data successfully read into the file {file_name}')

In [7]:
# Run merge data function
prem_file_name = 'E0_Data.csv'
cham_file_name = 'E1_Data.csv'
leag_file_name = 'E2_Data.csv'

prem_data_location = r'C:\Users\User\OneDrive - Heritage of God Ministries Society\Desktop\Project 1 WebScraping Final Wk\Premiership'
cham_data_location = r'C:\Users\User\OneDrive - Heritage of God Ministries Society\Desktop\Project 1 WebScraping Final Wk\Championship'
leag_data_location = r'C:\Users\User\OneDrive - Heritage of God Ministries Society\Desktop\Project 1 WebScraping Final Wk\League 1'

merge_csv(prem_data_location, prem_file_name)
merge_csv(cham_data_location, cham_file_name)
merge_csv(leag_data_location, leag_file_name)


Data successfully read into the file E0_Data.csv
Data successfully read into the file E1_Data.csv
Data successfully read into the file E2_Data.csv


##### Transformation

In [8]:
## Transformation function
def clean_data(base_directory, league, file_name):
    file_path = os.path.join(base_directory, league, file_name)

    # Read the csv and deal with the date issue - this also converts the date column to datetime datatype
    d_Data = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)

    d_Data['Date'] = pd.to_datetime(d_Data['Date'], format='%d/%m/%Y')

    # Convert the goal columns to integers
    d_Data[['FTAG', 'FTHG']] = d_Data[['FTAG', 'FTHG']].astype('int64')

    # Fill null times with a default time - 15:00
    d_Data['Time'] = d_Data['Time'].fillna('15:00')

    # Convert season column to str
    d_Data['season'] = d_Data['season'].astype(str)

    # Sort the data by date in descending order
    d_Data = d_Data.sort_values(by='Date', ascending=False)

    # Reset the index
    d_Data.reset_index(drop=True, inplace=True)

    return d_Data

In [9]:
# Run clean data function
base_directory = r'C:\Users\User\OneDrive - Heritage of God Ministries Society\Desktop\Project 1 WebScraping Final Wk'

prem_data = clean_data(base_directory, 'Premiership', 'E0_Data.csv')
cham_data = clean_data(base_directory, 'Championship', 'E1_Data.csv')
leag_data = clean_data(base_directory, 'League 1', 'E2_Data.csv')


C:\Users\User\AppData\Local\Temp\ipykernel_12416\1011505343.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d_Data = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12416\1011505343.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d_Data = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)
C:\Users\User\AppData\Local\Temp\ipykernel_12416\1011505343.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d_Data = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)


In [10]:
leag_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34176 entries, 0 to 34175
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Div       34176 non-null  object        
 1   season    34176 non-null  object        
 2   Date      34176 non-null  datetime64[ns]
 3   Time      34176 non-null  object        
 4   HomeTeam  34176 non-null  object        
 5   AwayTeam  34176 non-null  object        
 6   FTHG      34176 non-null  int64         
 7   FTAG      34176 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 2.1+ MB


In [11]:
cham_data.head()

,Div,season,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG
0,E1,2425,2024-10-20,15:00,Hull,Sunderland,0,1
1,E1,2425,2024-10-20,15:00,Hull,Sunderland,0,1
2,E1,2425,2024-10-19,12:30,Cardiff,Plymouth,5,0
3,E1,2425,2024-10-19,12:30,Cardiff,Plymouth,5,0
4,E1,2425,2024-10-19,12:30,Luton,Watford,3,0


#### Loading Data to the DB

In [12]:
load_dotenv(override=True)

True

In [13]:
# Credentials
hostname = os.getenv('hostname')
username = os.getenv('username')
password = os.getenv('password')
port = os.getenv('port')
db_name = os.getenv('db_name')

# Create the DB engine
db_url = f'postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{db_name}'
engine = create_engine(db_url)

In [21]:
### CREATE TABLE
## SPECIFY THE REQUIRED COLUMNS

schemaa = 'GoalBet'

my_columns = {
    'Div' : String,
    'Date' : DateTime,
    'Time' : String,
    'HomeTeam' : String,
    'AwayTeam' : String,
    'FTHG' : Integer,
    'FTAG' : Integer
}

In [22]:
## Loading the data to postgresql

prem_data.to_sql('Premiership', schema=schemaa, con=engine, index=False, dtype=my_columns)
cham_data.to_sql('Championship', schema=schemaa, con=engine, index=False, dtype=my_columns)
leag_data.to_sql('League_1', schema=schemaa, con=engine, index=False, dtype=my_columns)


176

##### Automation to get new data

In [23]:
## Get the last date from the DB
sql_query = f'SELECT max("Date") FROM "{schemaa}"."Premiership";'

last_date_record = pd.read_sql(sql_query, engine)

In [24]:
last_date = last_date_record.iloc[0, 0]

print(last_date)

2024-10-21 00:00:00
